**Preprocesamiento de la base de datos de Walmart**

Este archivo consta de los codigos y conclusiones de:
* Se obtiene series de tiempo
* Caracerizacion de la base de datos

# Carga de Librerias y Datos


*Se importan los módulos necesarios para trabajar*

In [1]:
#Pandas es utilizado para leer los set de datos
import pandas as pd
#Numpy es utilizado para generar las series de datos a graficar
import numpy as np
#Se importan modulos estadisticos para generar test de hipotesis, entre otros
from statsmodels.tsa.stattools import adfuller

#Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

*Se carga base de datos*

In [2]:
# Para acceder a los archivos del gdrive
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [3]:
cd /content/gdrive/MyDrive/Tesis/Datos

/content/gdrive/MyDrive/Tesis/Datos


In [4]:
df=pd.read_csv('df.csv')

In [5]:
df.shape

(380380, 20)

# Obtener series de Tiempo

*Se obtiene lista de dataframe ordenadados por Store y Dept*

**Para este punto de acorde al EDA realizado:**
* El departamento no 15, 53, 57, 61, 62, 63, 64, 66, 68, 69, 70, 73, 75, 76, 84, 86, 88, 89 no existe para ninguna tienda.
* Se comprueba no todas las semanas total estan presenten en cada Store y departamentos donde debe haber por serie 143 registros.

Para se filtra por aquellas datos vacios y por Store y Depts que no tengan la cantidad de regsitros totales, es decir **con datos faltantes**

In [ ]:
lista_dept

In [ ]:
series_time=[]
lista_Store=df.Store.unique()
lista_Store.sort()
lista_dept=df.Dept.unique()
lista_dept.sort()
serie_lista=[]
for i in lista_Store:
  for j in lista_dept:
    #lista=[]
    test=df[(df.Store==i) & (df.Dept==j)]
    if(test.empty!=True):
      #Se filtra por datos faltantes en la serie de tiempo
        series_time.append(test)
        x=test.to_numpy().tolist()
        serie_lista.append(x)

In [ ]:
#Largo de la la series
len(series_time)

2660

In [ ]:
series_time[15]

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Day,Week
15,1,17,2010-02-05,13223.76,False,-14.595679,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,A,151315,2010,2,5,5
82,1,17,2010-02-12,13403.66,True,-15.768519,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,A,151315,2010,2,12,6
149,1,17,2010-02-19,13485.61,False,-15.330247,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,A,151315,2010,2,19,7
216,1,17,2010-02-26,10667.06,False,-13.262346,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,A,151315,2010,2,26,8
283,1,17,2010-03-05,12657.65,False,-13.302469,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,A,151315,2010,3,5,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9261,1,17,2012-09-28,7155.75,False,-4.172840,3.666,3666.27,7.64,1.65,1417.96,4744.28,222.981658,6.908,A,151315,2012,9,28,39
9328,1,17,2012-10-05,9040.52,False,-6.496914,3.617,8077.89,0.00,18.22,3617.43,3626.14,223.181477,6.573,A,151315,2012,10,5,40
9395,1,17,2012-10-12,7908.36,False,-8.212963,3.601,2086.18,0.00,8.11,602.36,5926.45,223.381296,6.573,A,151315,2012,10,12,41
9462,1,17,2012-10-19,7640.43,False,-6.675926,3.594,950.33,0.00,4.93,80.25,2312.85,223.425723,6.573,A,151315,2012,10,19,42


**insight**: Se cuenta con un total de **2660** series de tiempo, se eliminan aquellas que no tengan la cantidad de muestras totales en el tiempo de la base de datos


---

#Caracterizar base de datos

Las caracteristicas a analizar son las siguientes:
* 1.Variables endógenas o exógenas
* 2.Variables univariantes o multivariantes
* 3.Serie continua o no continua
* 4.Serie estacionario o no estacionario
* 5.Serie desestructurada o estructurada
* 6.Complejidad

1.**Variables endógenas o exógenas**:

Para determinar estas variables, el investigador debe analizar y decidir cuales y cuantas existen. A continuación se expone la justificación y las elecciones: Se considera que la varible endogena corresponde a la variable objetivo "Weekly_Sales". Todas las demás corresponden a varibles exogenas. De esta manera, se tiene 1 variable endogena y 18 variables exogenas que son:

* Store
* Dept
* IsHoliday
* Temperature
* Fuel_Price
* MarkDown1
* MarkDown2
* MarkDown3
* MarkDown4
* MarkDown5
* CPI
* Unemployment
* Type
* Size
* Year
* Month
* Day
* Week

2.**Variables univariantes o multivariantes:**

Para determinar estas variables, el investigador debe analizar y decidir cuales y cuantas existen. A continuación se expone la justificación y las elecciones: El problema claramente es multivariante, pero para ser más precisos, se utilizarán 18 variables de entrada y una variable de salida. De esta manera, es problema es multivariable en las entradas y univariante en la salida.

3.**Series con muestreo regular o irregular:**

Para determinar esta caracteristica, el investigador debe analizar y decidir si la serie es continua o no. A continuación se expone la justificación y la elección: Se considera que la serie claramente tiene un muestre regular, ya que no existen mediciones faltantes, segun el EDA realizado.

4.**Serie estacionario o no estacionario:**

Comprobar la estacionariedad de este problema, se realiza la prueba estadistica "Augmented Dickey-Fuller Test (ADF)". El nivel de significancia usado sera de 1%. De esta manera, se contabiliza cuantas series son estacionarias y cuantas no, de tal manera que si si la mayoria de las series son estacionaria, el problema se considera estacionario, analogo para el caso de no estacionariedad

Prueba Dickey Fuller

* H0 = serie de tiempo posee raíz unitaria y no es estacionaria.

* H1 = serie de tiempo no posee raíz unitaria y es estacionaria.
si el valor P en la prueba Dickey Fuller es menor que el nivel de significación (0.01) se rechaza la hipótesis nula.



*Se realiza test de  Dickey Fuller a cada muestra de series de tiempo por separado, teniedo un analisis de 2660 muestras de series de tiempo*

In [ ]:
resultado=[]
estacionario=1
no_estacionario=0
for serie in series_time:
  stat, p, lags, obs, crit, t = adfuller(serie["Weekly_Sales"])
  #print('stat=%.3f, p=%.3f' % (stat, p))
  if p > 0.05:
    #print('Probably not Stationary')
    resultado.append(estacionario)
  else:
    #print('Probably Stationary')
    resultado.append(no_estacionario)

In [ ]:
aux=0
aux_2=0
for i in resultado:
  if(i==1):
    aux=aux+1
  else:
    aux_2=aux_2+1

print(f"Se tiene {aux} series estacionarias")
print(f"Se tiene {aux_2} NO series estacionarias")

Se tiene 698 series estacionarias
Se tiene 1962 NO series estacionarias


**insight**: Al haber mas series no estacionarias, se concluye que este problemas es no estacionario.


---

5.**Serie desestructurada o estructurada:**


6.**Complejidad**


In [ ]:
pip install MFDFA

In [ ]:
from MFDFA import MFDFA
from MFDFA import fgn

In [ ]:
# Seleccione una banda de retrasos, que son enteros
lag = np.array([1,2,3,4,5,6,7,8,9,10,12,14,18,20,25,30,35,40])

resul=[]

#Seleccione una lista de poderes q
q = [1, 2, 5, 8, 10]

#El orden del ajuste del polinomio
order = 2

#MFDFA:
for serie in series_time:
  lag, dfa = MFDFA(serie["Weekly_Sales"].values, lag = lag, q = q, order = order)
  H = np.polyfit(np.log(lag[:]), np.log(dfa[:]),1)[0]
  mediaAux = round(np.mean(H),2)
  resul.append(mediaAux)

In [ ]:
import statistics
mean_MFDFA = float(statistics.mean(resul))

print(f"El MFDFA como promedio es: {mean_MFDFA}")

El MFDFA como promedio es: 0.8405488721804512


#Conclusiones